In [ ]:
import numpy as np
import pandas as pd

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
!pip install unsloth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 4.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.8/299.8 kB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 504.6/504.6 kB 41.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.7/166.7 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.1/117.1 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.2/821.2 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.7/155.7 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 122.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/2

In [ ]:
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

Found existing installation: unsloth 2025.8.1
Uninstalling unsloth-2025.8.1:
  Successfully uninstalled unsloth-2025.8.1
  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-req-build-tpa63ov5
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-req-build-tpa63ov5
  Resolved https://github.com/unslothai/unsloth.git to commit a78b86e5c9c08b90f53a4ef89e6b9c6860fe66dc
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for unsloth: filename=unsloth-2025.8.1-py3-none-any.whl size=300335 sha256=4a685eaa0cedffd9a308deba4670f233b4787e693b678b860721263f72c7c307
  Stored in directory: /tmp/pip-ephem-wheel-cache-de9v7mem/wheels/d1/17/05/850ab10c33284a4763b0595cd8ea9d01fce6e221cac24b3c01
Successfully built unsloth


In [ ]:
from unsloth import FastLanguageModel
import torch

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/DeepSeek-R1-Distill-Llama-8B-unsloth-bnb-4bit",
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.8.1: Fast Llama patching. Transformers: 4.54.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.31.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/236 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

## Setup Parameter-Efficient Fine-Tuning (PEFT)
The **FastLanguageModel.get_peft_model** function modifies a pre-trained language model to use PEFT techniques, which allow fine-tuning of the model using fewer resources and parameters.

The method introduces additional tunable parameters (like LoRA matrices) to specific layers of the model while freezing most of the original model weights.

***In this Project, we use LORA to fine-tune the DeepSeek R1 LLM.***

First, it takes the existing model. Then, it applies a technique called PEFT (Parameter-Efficient Fine-Tuning).  Think of it as adding small, adjustable "knobs" instead of changing the whole engine.

r=4 and lora_alpha=16 are just settings for how many "knobs" and how sensitive they are.  target_modules specifies where these knobs are attached – specifically, parts of the model that handle questions, keys, values, and outputs.  lora_dropout=0 means no "knobs" are randomly turned off during training.

bias="none" means no extra adjustments to the model's biases. use_gradient_checkpointing="unsloth" is a memory-saving trick for training. random_state=42 ensures we get the same results if we run this again.  use_rslora=False and loftq_config=None are more advanced settings that are turned off here.

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 4,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 42,
    use_rslora = False,
    loftq_config = None,
)

Not an error, but Unsloth cannot patch MLP layers with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Unsloth 2025.8.1 patched 32 layers with 32 QKV layers, 32 O layers and 0 MLP layers.


## Load Dataset

The vicgalle/alpaca-gpt4 dataset is a collection of 52,000 instruction-following instances designed to fine-tune language models (LLMs). It was created by Vic Galie and is available on Hugging Face.

In [ ]:
from datasets import load_dataset
dataset = load_dataset("vicgalle/alpaca-gpt4", split = "train")
print(dataset.column_names)

README.md: 0.00B [00:00, ?B/s]

(…)-00000-of-00001-6ef3991c06080e14.parquet:   0%|          | 0.00/48.4M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/52002 [00:00<?, ? examples/s]

['instruction', 'input', 'output', 'text']


In [ ]:
dataset[0]

{'instruction': 'Give three tips for staying healthy.',
 'input': '',
 'output': '1. Eat a balanced and nutritious diet: Make sure your meals are inclusive of a variety of fruits and vegetables, lean protein, whole grains, and healthy fats. This helps to provide your body with the essential nutrients to function at its best and can help prevent chronic diseases.\n\n2. Engage in regular physical activity: Exercise is crucial for maintaining strong bones, muscles, and cardiovascular health. Aim for at least 150 minutes of moderate aerobic exercise or 75 minutes of vigorous exercise each week.\n\n3. Get enough sleep: Getting enough quality sleep is crucial for physical and mental well-being. It helps to regulate mood, improve cognitive function, and supports healthy growth and immune function. Aim for 7-9 hours of sleep each night.',
 'text': 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nGive three tips for 

Let's format the dataset in a way suitable for conversational AI training using the ShareGPT format, which is designed for multi-turn conversations.

In [ ]:
from unsloth import to_sharegpt

dataset = to_sharegpt(
    dataset,
    merged_prompt = "{instruction}[[\nYour input is:\n{input}]]",
    output_column_name = "output",
    conversation_extension = 3, # Select more to handle longer conversations
)

Merging columns:   0%|          | 0/52002 [00:00<?, ? examples/s]

Converting to ShareGPT:   0%|          | 0/52002 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/52002 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/52002 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/52002 [00:00<?, ? examples/s]

Extending conversations:   0%|          | 0/52002 [00:00<?, ? examples/s]

In [ ]:
from unsloth import standardize_sharegpt
dataset = standardize_sharegpt(dataset)

Unsloth: Standardizing formats (num_proc=2):   0%|          | 0/52002 [00:00<?, ? examples/s]

In [ ]:
dataset[0]['conversations']

[{'content': 'Give three tips for staying healthy.', 'role': 'user'},
 {'content': '1. Eat a balanced and nutritious diet: Make sure your meals are inclusive of a variety of fruits and vegetables, lean protein, whole grains, and healthy fats. This helps to provide your body with the essential nutrients to function at its best and can help prevent chronic diseases.\n\n2. Engage in regular physical activity: Exercise is crucial for maintaining strong bones, muscles, and cardiovascular health. Aim for at least 150 minutes of moderate aerobic exercise or 75 minutes of vigorous exercise each week.\n\n3. Get enough sleep: Getting enough quality sleep is crucial for physical and mental well-being. It helps to regulate mood, improve cognitive function, and supports healthy growth and immune function. Aim for 7-9 hours of sleep each night.',
  'role': 'assistant'},
 {'content': 'Describe what a monotheistic religion is.', 'role': 'user'},
 {'content': 'A monotheistic religion is a type of relig

## Customizable Chat Templates

This code takes a dataset, formats it into a chat-friendly format, and then applies a template so that the AI can understand the instructions and responses correctly.

In [ ]:
chat_template = """Below are some instructions that describe some tasks. Write responses that appropriately complete each request.

### Instruction:
{INPUT}

### Response:
{OUTPUT}"""

from unsloth import apply_chat_template
dataset = apply_chat_template(
    dataset,
    tokenizer = tokenizer,
    chat_template = chat_template,
    # default_system_message = "You are a helpful assistant", << [OPTIONAL]
)

Unsloth: We automatically added an EOS token to stop endless generations.


Map:   0%|          | 0/52002 [00:00<?, ? examples/s]

## Train the model

Let's define and configure a fine-tuning trainer for a language model using the **SFTTrainer** class from the **trl** library (likely for fine-tuning language models) and transformers' **TrainingArguments**

It uses SFTTrainer from the trl library, which is specifically for Supervised Fine-Tuning.  This means the model will learn from the dataset of instructions and responses you prepared earlier.

It takes the model (the tuned language model), the tokenizer (for breaking down text), and the dataset. dataset_text_field="text" tells it where the actual text is in the dataset.  max_seq_length=2048 limits the length of text sequences the model processes at once. dataset_num_proc=2 uses two processes to prepare the data, and packing=False disables a specific data packing technique.

Then, it configures the training process with TrainingArguments.  per_device_train_batch_size=2 means each training device (like a GPU) will process 2 examples at a time. gradient_accumulation_steps=4 combines the gradients from 4 batches to simulate a larger batch size. warmup_steps=5 gradually increases the learning rate at the beginning. max_steps=20 limits the total training steps. learning_rate=2e-4 sets the learning rate.  fp16 and bf16 control the precision of calculations (using either half-precision or bfloat16 if supported, for faster training). logging_steps=1 logs training progress every step. optim="adamw_8bit" specifies the optimizer. weight_decay=0.01 is a regularization technique. lr_scheduler_type="linear" sets how the learning rate changes over time. seed=3407 ensures reproducibility. output_dir="outputs" specifies where to save the trained model. report_to="none" disables reporting to services like WandB (Weights & Biases).


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = 2048,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    warmup_steps=5,
    max_steps=20,
    learning_rate=2e-4,
    fp16=not is_bfloat16_supported(),
    bf16=is_bfloat16_supported(),
    logging_steps=1,
    optim="adamw_8bit",
    weight_decay=0.01,
    lr_scheduler_type="linear",
    seed=3407,
    output_dir="outputs",
    report_to="none",
    )
)

Unsloth: Tokenizing ["text"]:   0%|          | 0/52002 [00:00<?, ? examples/s]

The trainer orchestrates the fine-tuning process by combining the model, tokenizer, dataset, and hyperparameters. It ensures efficient training, taking advantage of mixed precision (fp16 or bf16) and memory-efficient optimizations (e.g., AdamW with 8-bit precision). Additionally, it handles sequence preprocessing, gradient accumulation, and learning rate scheduling.

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 52,002 | Num Epochs = 1 | Total steps = 20
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 8 x 1) = 8
 "-____-"     Trainable parameters = 3,407,872 of 8,033,669,120 (0.04% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.935800
2,1.938200
3,1.850200
4,2.038300
5,2.135300
6,1.818200
7,1.784900
8,1.706600


Step,Training Loss
1,1.935800
2,1.938200
3,1.850200
4,2.038300
5,2.135300
6,1.818200
7,1.784900
8,1.706600
9,1.744800
10,1.568300


## Ollama

OLAMA is a powerful tool that enables you to run large language models (LLMs) directly on your own computer (laptop or desktop).


In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


Let's save this model.

In [ ]:
model.save_pretrained_gguf("model", tokenizer)

Unsloth: ##### The current model auto adds a BOS token.
Unsloth: ##### Your chat template has a BOS token. We shall remove it temporarily.
Unsloth: You have 1 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which might take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 6.0G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 5.06 out of 12.67 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


 38%|███▊      | 12/32 [00:00<00:00, 25.75it/s]
We will save to Disk and not RAM now.
100%|██████████| 32/32 [02:52<00:00,  5.40s/it]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model/pytorch_model-00001-of-00004.bin...
Unsloth: Saving model/pytorch_model-00002-of-00004.bin...
Unsloth: Saving model/pytorch_model-00003-of-00004.bin...
Unsloth: Saving model/pytorch_model-00004-of-00004.bin...
Done.


Unsloth: Converting llama model. Can use fast conversion = False.


==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits might take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q8_0'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: CMAKE detected. Finalizing some steps for installation.
Unsloth: [1] Converting model at model into q8_0 GGUF format.
The output location will be /content/model/unsloth.Q8_0.gguf
This might take 3 minutes...
INFO:hf-to-gguf:Loading model: model
INFO:hf-to-gguf:Model architecture: LlamaForCausalLM
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:rope_freqs.weight,           torch.float32 --> F32, shape = {64}
INFO:hf-to-gguf:gguf: loading model weight map from 'pytorch_model.bin.index.json'
INFO:hf-to-gguf:gguf:

Unsloth: ##### The current model auto adds a BOS token.
Unsloth: ##### We removed it in GGUF's chat template for you.


Unsloth: Conversion completed! Output location: /content/model/unsloth.Q8_0.gguf
Unsloth: Saved Ollama Modelfile to model/Modelfile


# Start the OLLAMA Server

In [ ]:
import subprocess
subprocess.Popen(["ollama", "serve"])
import time
time.sleep(3)
print(tokenizer._ollama_modelfile)

FROM {__FILE_LOCATION__}

TEMPLATE """Below are some instructions that describe some tasks. Write responses that appropriately complete each request.{{ if .Prompt }}

### Instruction:
{{ .Prompt }}{{ end }}

### Response:
{{ .Response }}<｜end▁of▁sentence｜>"""

PARAMETER stop "<|finetune_right_pad_id|>"
PARAMETER stop "<｜User｜>"
PARAMETER stop "<|start_header_id|>"
PARAMETER stop "<|python_tag|>"
PARAMETER stop "<｜Assistant｜>"
PARAMETER stop "<|eot_id|>"
PARAMETER stop "<think>"
PARAMETER stop "<｜▁pad▁｜>"
PARAMETER stop "<|end_header_id|>"
PARAMETER stop "<|eom_id|>"
PARAMETER stop "</think>"
PARAMETER stop "<｜end▁of▁sentence｜>"
PARAMETER stop "<|reserved_special_token_"
PARAMETER temperature 1.5
PARAMETER min_p 0.1


This command registers the fine-tuned model (deepseek_finetuned_model) with Ollama. Once registered:

- The model can be run locally using Ollama.
- It becomes accessible for further tasks, such as querying, evaluating, or deploying in specific applications.
- Ollama ensures the model is formatted and stored correctly for efficient usage

In [ ]:
!ollama create deepseek_finetuned_model -f ./model/Modelfile

Error: ollama server not responding - could not connect to ollama server, run 'ollama serve' to start it


In [ ]:
!pip install ollama

In [ ]:
from IPython.display import Markdown
import ollama

response = ollama.chat(model="deepseek_finetuned_model",
                       messages=[{"role": "user",
                                  "content": "How to add chart to a document?"},
                      ])

Markdown(response.message.content)

1. Open the Word document or other text processor where you want to insert the chart.
2. Click on the location where you want to place the chart.
3. Choose the chart option from the menu bar and select 'Insert Chart' or choose an existing template if desired.

You can also use the following steps for more information:

1. Open Excel and save your work so that all data is saved correctly.

2. Select a range of cells in which you want to place the chart.

3. Click on the chart wizard icon in the toolbar.

4. Select 'insert chart' option from the list.

5. Select a chart type, specify the location where the chart will be placed, and click the OK button.

6. Now your document includes the chart as specified.

To add a chart to a document, follow these steps:

- Insert a Table: Start by inserting a table into the document. You can do this using the 'Table' tool in most word processors.
- Insert Data: Add data into the table. Ensure that your data is properly formatted and organised before adding the chart.
- Choose a Chart Type: Select the type of chart you want to create from the available options (e.g., bar chart, pie chart, line graph, etc.).
- Edit the Chart Data: Add the necessary data points and formatting to the chart using the chart editor that appears once the chart is selected.
- Format the Table and Chart Together: Make sure the table and chart work well together by adjusting alignment, spacing, and other design elements as needed.

For more detailed instructions, you may want to consult a guide or use a tool such as Microsoft Word's chart features.

In [ ]:
from IPython.display import Markdown
import ollama

response = ollama.chat(model="deepseek_finetuned_model",
                       messages=[{"role": "user",
                                  "content": "سه شهر پرجمعیت آمریکا رو نام ببر "},
                      ])

Markdown(response.message.content)

واشیانگتن، نیویورک و لس آنجلس

In [ ]:
from IPython.display import Markdown
import ollama

response = ollama.chat(model="deepseek_finetuned_model",
                       messages=[{"role": "user",
                                  "content": "پایتخت ژاپن کجاست؟"},
                      ])

Markdown(response.message.content)

پایتخت ژاپن کجاست؟ پایتخت ژاپن توی شهر توکیو است.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
